In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import os
import itertools
import os, stat, time
from os.path import dirname as up
import shutil
import glob
from PIL import Image

In [3]:
from glob import glob
data = glob('../input/cassava-leaf-disease-classification/train_images/*.jpg', recursive = True)

In [4]:
len(data)

In [5]:
import cv2
import matplotlib.pyplot as plt

for i in data[:5]:
    img = cv2.imread(i)
    img_1 = cv2.resize(img, (200, 200))
    plt.imshow(img_1, cmap = 'binary')
    plt.show()

In [6]:
img_size = 70

In [ ]:
import cv2
import matplotlib.pyplot as plt

images = list()
labels = list()

for i in data:
    if i.endswith('.jpg'):
        label = i[-5]
        img = cv2.imread(i)
        img_1 = cv2.resize(img, (img_size, img_size))
        images.append(img_1)
        labels.append(label)

In [ ]:
len(images)

In [ ]:
from numpy import array

a = array(images)
a.shape

In [ ]:
x = np.stack(images)

In [ ]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(labels)

In [ ]:
x = x / 255

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 42, test_size = 0.25)

In [ ]:
print('len(x_train):', len(x_train))
print('len(x_test):', len(x_test))
print('len(y_train):', len(y_train))
print('len(y_test):', len(y_test))

In [ ]:
print('x_train', x_train.shape)
print('x_test', x_test.shape)
print('y_train', y_train.shape)
print('y_test', y_test.shape)

# Building the model

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Flatten, RepeatVector
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers
from tensorflow.keras.regularizers import L2
from tensorflow.keras.applications.efficientnet import EfficientNetB2

def nn_model():
    model = models.Sequential()

    # model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
    # model.add(layers.MaxPool2D(2, 2))
    # model.add(layers.Flatten())
    # model.add(layers.Conv2D(256, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
    # model.add(layers.MaxPool2D(2, 2))
    # model.add(layers.Flatten())
    # model.add(layers.Conv2D(256, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
    # model.add(layers.MaxPool2D(2, 2))

    model.add(EfficientNetB2(include_top = False, weights = 'imagenet', input_shape = (img_size, img_size, 3)))
    
    # model.add(layers.Flatten())
    
    # model.add(layers.Dropout(0.4))
    # model.add(layers.Dense(128, activation = 'relu', kernel_regularizer = L2(0.00001)))
    # model.add(layers.Dense(128, activation = 'relu', kernel_regularizer = L2(0.00001)))

    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(10, activation='softmax'))

    return model

model = nn_model()
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

opt = Adam(lr = 0.0001, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad =  False)

model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Training the Model

In [ ]:
history = model.fit(x, y, epochs = 25, validation_split = 0.4, batch_size = 64)

# Evaluating the model

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)

In [ ]:
plt.figure(figsize = (12, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['train_data', 'test_data'])
plt.title('Loss Analysis')
plt.show()

In [ ]:
plt.figure(figsize = (12, 5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(['train_data', 'test_data'])
plt.title('Accuracy Analysis')
plt.show()